# copybuffer

### Copyright Information

In [ ]:
# Copyright (c) 2024-2025 Ben Ashbaugh
#
# SPDX-License-Identifier: MIT

## Sample Purpose

This is first example that uses OpenCL APIs to do work. In this very simple sample, OpenCL APIs are used to copy the contents of one buffer to another buffer on the OpenCL device. To do this, OpenCL APIs are used to create both buffers, to create the OpenCL command queue, and to initialize the source buffer and verify the contents of the destination buffer on the host.

## Sample

The first thing we will do is to import pyopencl so we have access to OpenCL from Python.
We will also import numpy.

In [ ]:
import numpy as np
import pyopencl as cl

By default, this sample will run in the first enumerated OpenCL device on the first enumerated OpenCL platform.
To choose a different OpenCL platform, simply change the platform index or device index to a different value.

In [ ]:
if __name__ == "__main__":
    platformIndex = 0
    deviceIndex = 0

    platforms = cl.get_platforms()
    print('Running on platform: ' + platforms[platformIndex].get_info(cl.platform_info.NAME))

    devices = platforms[platformIndex].get_devices()
    print('Running on device: ' + devices[deviceIndex].get_info(cl.device_info.NAME))

In order to create OpenCL objects we first need to create an OpenCL _context_.
An OpenCL context describes the state of an OpenCL application.
Most OpenCL objects created against one context cannot be used in a different context.

To create an OpenCL context we must pass the set of OpenCL devices in the context.
It is most common to create an OpenCL context for a single OpenCL device, but a context can also be created for multiple OpenCL devices if the devices are in the same OpenCL platform.
In this example we are going to create an OpenCL context for a single OpenCL device.

In [ ]:
    context = cl.Context([devices[deviceIndex]])

Now that we have a context we can create an OpenCL command queue.
An OpenCL command queue is the way to submit work to an OpenCL device.
To create an OpenCL command queue we must pass the context that the command queue will be created in, and the OpenCL device that the command queue will submit work to.
We need to pass the OpenCL device becauase the OpenCL context may be created for multiple OpenCL devices.

In [ ]:
    commandQueue = cl.CommandQueue(context, devices[deviceIndex])

In this example we are going to copy data from one OpenCL buffer to a different OpenCL buffer, on the OpenCL device.
To do this we need to create a source buffer to copy from and a destination buffer to copy to.
By default we will copy one million integers, but this can be changed by modifying the constant below.

In [ ]:
    numElems = 1024 * 1024
    deviceMemSrc = cl.Buffer(context, cl.mem_flags.ALLOC_HOST_PTR, numElems * np.uint32().itemsize)
    deviceMemDst = cl.Buffer(context, cl.mem_flags.ALLOC_HOST_PTR, numElems * np.uint32().itemsize)

There are several ways to modify the data in an OpenCL buffer, but one of the most common ways is to map the buffer so it is accessible on the host.
We will map the source buffer to initialize its contents.
Mapping the buffer uses the command queue we created earlier.

In [ ]:
    mapped_src, event = cl.enqueue_map_buffer(commandQueue, deviceMemSrc,
                                              cl.map_flags.WRITE_INVALIDATE_REGION,
                                              0, numElems, np.uint32)
    with mapped_src.base:
        for i in range(numElems):
            mapped_src[i] = i

With the source buffer initialized, we can finally copy its contents to the destination buffer.
Copying memory is a common operation so there is a dedicated OpenCL function to perform the copy.
In a subsequent sample we will explore how to do the copy ourselves, instead.
For now though, we will simply call the OpenCL function to perform the copy.
Because the copy is executing on the OpenCL device it also uses the command queue we created earlier.

In [ ]:
    event = cl.enqueue_copy(commandQueue, deviceMemDst, deviceMemSrc)

All that remains now is to verify that the copy succeeded!
To do this, we will map the destination buffer, and check that it has the same data we used to initilaize the source buffer.

In [ ]:
    mapped_dst, event = cl.enqueue_map_buffer(commandQueue, deviceMemDst,
                                              cl.map_flags.READ,
                                              0, numElems, np.uint32)
    with mapped_dst.base:
        mismatches = 0
        for i, val in enumerate(mapped_dst):
            if val != i:
                if mismatches < 16:
                    print('Mismatch!  dst[{}] == {}, want {}'.format(i, val, i))
                mismatches = mismatches + 1
        if mismatches != 0:
            print('Error: Found {} mismatches / {} values!!!'.format(mismatches, numElems))
        else:
            print('Success.')

If the copy executed correctly we expect to find zero mismatching elements.